<a href="https://colab.research.google.com/github/ExpectoX1/Extractive_Haystack_QA/blob/main/ExtractiveMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'farm-haystack[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!pip install tensorflow_probability==0.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 36.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow_probability
    Found existing installation: tensorflow-probability 0.12.2
    Uninstalling tensorflow-probability-0.12.2:
      Successfully uninstalled tensorflow-probability-0.12.2


In [10]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING) #Logging on the console (helper)
logging.getLogger("haystack").setLevel(logging.INFO)

In [9]:
from haystack.document_stores import InMemoryDocumentStore
#Haystack Document store is used here, helps to arrange the data being downloaded from the URL.

document_store = InMemoryDocumentStore(use_bm25=True)
from haystack.utils import fetch_archive_from_http

doc_dir = "data/test1" # path

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip", # dataset being imported
    output_dir=doc_dir
)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt1.zip to 'data/test1'


True

In [11]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline # indexing the text accordingly

files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index) # ideally just appending all the files into one big file.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.


Converting files:   0%|          | 0/183 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/183 [00:00<?, ?docs/s]

Updating BM25 representation...:   0%|          | 0/2356 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': '\n"\'\'\'Power Is Power\'\'\'" is a song by American singer SZA, Canadian singer The Weeknd and American rapper Travis Scott. It was released on April 18, 2019, as a single from the \'\'Game of Thrones\'\' companion soundtrack, \'\'For the Throne\'\'. The song is the second of the two collaborative singles released by The Weeknd and Scott in 2019, being preceded by "Wake Up".\n\n==Background and release==\nFollowing the release of the collaboration "Wake Up" by Travis Scott featuring The Weeknd, as the fourth single from Scott\'s \'\'Astroworld\'\' on March 26, 2019. In early April of that year, rumours regarding the possibility of a \'\'Game of Thrones\'\' inspired track by SZA, The Weeknd, and Travis Scott began to circulate around the web. The artists\' involvement on the \'\'For the Throne\'\' soundtrack was then later confirmed on April 8, via an announcement of the soundtrack by Columbia Records. Following the release of the promotional sing

In [12]:
from haystack.nodes import BM25Retriever #BM25 Helps in fast retrieval of the documents.

retriever = BM25Retriever(document_store=document_store)
from haystack.nodes import FARMReader # saves the model for futher instances.

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True) #Model being used.

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [13]:
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever) # using a pipeline called ExtractiveQA , a pipeline present in haystack which connects the Reader and Retriever

In [14]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True) # Init again

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [15]:
prediction = pipe.run(
    query=input(),
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
) # code to run the execution, top_k (Retriever) represents number of answers needed , top_k (Reader) represents number of contexts to scan for the answer.

what has received widespread praise from critics


Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

In [16]:
from pprint import pprint

pprint(prediction) # pretty print the prediction by the model.

{'answers': [<Answer {'answer': "Tyrion's storyline", 'type': 'extractive', 'score': 0.7198374271392822, 'context': "ed Sand Snakes were maligned by critics and fans alike, as many found them cheesy and unlikeable. In contrast, Tyrion's storyline was highly praised. ", 'offsets_in_document': [{'start': 1218, 'end': 1236}], 'offsets_in_context': [{'start': 111, 'end': 129}], 'document_ids': ['d3a6fcb0953d81ea9e6d72ad633fc0ce'], 'meta': {'_split_id': 11}}>,
             <Answer {'answer': 'Book of the Stranger', 'type': 'extractive', 'score': 0.6399433612823486, 'context': 'to have them released; and Daenerys Targaryen faces the khals.\n\n"Book of the Stranger" received widespread acclaim from critics, who noted the reunion', 'offsets_in_document': [{'start': 631, 'end': 651}], 'offsets_in_context': [{'start': 65, 'end': 85}], 'document_ids': ['4f9ae30c63890f4a91c27586e7425074'], 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'The Spoils of War', 'type': 'extractive', 'score